In [1]:
import pandas as pd
# Tạo dữ liệu giả lập cho các cột 'product', 'amount', và 'date'
data = pd.DataFrame({
    'product': ['A', 'B', 'C', 'A', 'B', 'C', 'A', 'B', 'C', 'A'],
    'amount': [100, 200, 300, 150, 250, 350, 200, 300, 400, 500],
    'date': ['2024-01-15', '2024-01-15', '2024-02-15', '2024-02-15', '2024-03-15','2024-03-15', '2024-04-15', '2024-04-15', '2024-05-15', '2024-05-15']
})
# Lưu dữ liệu vào file CSV để sử dụng cho các bước ETL
data.to_csv('data.csv', index=False)

In [3]:
!pip install pandas sqlalchemy sqlite3

ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3


In [4]:
!pip install sqlalchemy

In [5]:
from sqlalchemy import create_engine
import pandas as pd
# Trích xuất dữ liệu từ một file CSV
data = pd.read_csv('data.csv')
print(data.head())

  product  amount        date
0       A     100  2024-01-15
1       B     200  2024-01-15
2       C     300  2024-02-15
3       A     150  2024-02-15
4       B     250  2024-03-15


In [6]:
# Loại bỏ các hàng có giá trị thiếu
data.dropna(inplace=True)
# Chuyển đổi kiểu dữ liệu
data['amount'] = data['amount'].astype(float)

In [7]:
# Tạo kết nối đến SQLite
engine = create_engine('sqlite:///warehouse.db')
data.to_sql('sales_data', engine, if_exists='replace', index=False)

10

In [8]:
# Tính tổng doanh thu theo từng sản phẩm
aggregated_data = data.groupby('product')['amount'].sum().reset_index()
print(aggregated_data)

  product  amount
0       A   950.0
1       B   750.0
2       C  1050.0


In [9]:
# Tính doanh thu theo từng sản phẩm và từng tháng
data['month'] = pd.to_datetime(data['date']).dt.month
olap_result = data.pivot_table(values='amount', index='product',
columns='month', aggfunc='sum')
print(olap_result)

month        1      2      3      4      5
product                                   
A        100.0  150.0    NaN  200.0  500.0
B        200.0    NaN  250.0  300.0    NaN
C          NaN  300.0  350.0    NaN  400.0


In [10]:
# Kết hợp tổng hợp và pivot table
monthly_sales = data.groupby(['product', 'month'])['amount'].sum().unstack()
print(monthly_sales)
# Lưu kết quả vào một bảng mới trong SQLite
monthly_sales.to_sql('monthly_sales_report', engine, if_exists='replace')

month        1      2      3      4      5
product                                   
A        100.0  150.0    NaN  200.0  500.0
B        200.0    NaN  250.0  300.0    NaN
C          NaN  300.0  350.0    NaN  400.0


3